# **Projeto PCD: K-Means 1D - Etapa 3 (MPI)**

Implementação distribuída usando **MPI (Message Passing Interface)**.

**Objetivo:** Distribuir o processamento dos pontos entre múltiplos processos para obter speedup em memória distribuída.

In [ ]:
# 1. INSTALAÇÃO DO MPI (MPICH)
!apt-get update
!apt-get install -y mpich

In [ ]:
# 2. GERAÇÃO DE DADOS (Mesma lógica das etapas anteriores)
import numpy as np

N = 1000000  # 1 Milhão de pontos (ajuste conforme necessário)
K = 16

print(f"Gerando N={N} pontos e K={K} centroides...")
initial_centroids = np.linspace(5, K*10 - 5, K) + np.random.rand(K) * 2 - 1
np.savetxt("centroides_iniciais.csv", initial_centroids, fmt='%.6f')

points_per_cluster = N // K
all_points = []
for i in range(K):
    cluster_points = np.random.normal(loc=initial_centroids[i], scale=2.0, size=points_per_cluster)
    all_points.append(cluster_points)

remaining = N - (points_per_cluster * K)
if remaining > 0:
    all_points.append(np.random.normal(loc=initial_centroids[0], scale=2.0, size=remaining))

final_data = np.concatenate(all_points)
np.random.shuffle(final_data)
np.savetxt("dados.csv", final_data, fmt='%.6f')
print("Dados gerados.")

In [ ]:
# 3. COMPILAÇÃO
!mpicc -O2 kmeans_1d_mpi.c -o kmeans_1d_mpi -lm
print("Compilação concluída.")

In [ ]:
# 4. EXECUÇÃO E EXPERIMENTOS
import subprocess
import re
import matplotlib.pyplot as plt

def run_mpi(n_procs):
    # Adicionando --oversubscribe para evitar erros em ambientes com poucos cores
    cmd = [
        "mpirun", "--allow-run-as-root", "--oversubscribe", "-np", str(n_procs),
        "./kmeans_1d_mpi", "dados.csv", "centroides_iniciais.csv", "50", "0.000001"
    ]
    # Capture both stdout and stderr
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Parse Output
    time_ms = 0.0
    time_comm = 0.0
    sse = 0.0
    
    # Regex atualizado para capturar Time_Comm
    match = re.search(r"FINAL_DATA: Time=([\d\.]+) Time_Comm=([\d\.]+) SSE=([\d\.]+)", result.stdout)
    if match:
        time_ms = float(match.group(1))
        time_comm = float(match.group(2))
        sse = float(match.group(3))
    else:
        # Fallback para formato antigo se não recompilou
        match_old = re.search(r"FINAL_DATA: Time=([\d\.]+) SSE=([\d\.]+)", result.stdout)
        if match_old:
            time_ms = float(match_old.group(1))
            sse = float(match_old.group(2))
        else:
            # Se falhar, imprime o erro para debug
            print(f"ERRO ao rodar com P={n_procs}:")
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)
        
    return time_ms, time_comm, sse

procs_list = [1, 2, 3, 4]
times = []
comm_times = []
sses = []

print("Iniciando experimentos MPI...")
for p in procs_list:
    t, tc, s = run_mpi(p)
    times.append(t)
    comm_times.append(tc)
    sses.append(s)
    print(f"P={p}: Tempo={t:.1f} ms, Comm={tc:.1f} ms, SSE={s:.6f}")

# Plot Speedup (Evita divisão por zero)
if len(times) > 0 and times[0] > 0:
    speedups = []
    valid_procs = []
    for i, t in enumerate(times):
        if t > 0:
            speedups.append(times[0]/t)
            valid_procs.append(procs_list[i])
    
    if speedups:
        plt.figure(figsize=(12, 5))
        
        # Subplot 1: Speedup
        plt.subplot(1, 2, 1)
        plt.plot(valid_procs, speedups, 'o-', label='Speedup')
        plt.plot(valid_procs, valid_procs, 'k--', alpha=0.3, label='Ideal')
        plt.title("Speedup MPI")
        plt.xlabel("Número de Processos")
        plt.ylabel("Speedup")
        plt.legend()
        plt.grid(True)
        
        # Subplot 2: Tempo Total vs Comunicação
        plt.subplot(1, 2, 2)
        plt.plot(valid_procs, [times[i] for i in range(len(valid_procs))], 'o-', label='Total Time')
        plt.plot(valid_procs, [comm_times[i] for i in range(len(valid_procs))], 'x--', label='Comm Time')
        plt.title("Tempo Total vs Comunicação")
        plt.xlabel("Número de Processos")
        plt.ylabel("Tempo (ms)")
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
else:
    print("Não foi possível gerar o gráfico de speedup (Baseline falhou ou tempo zero).")